In [ ]:
!pip install langchain openai

In [ ]:
# Azure OpenAI
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_KEY = ""
DEPLOYMENT_NAME = "gpt-4o"

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain.chat_models import AzureChatOpenAI

In [ ]:
llm = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_KEY,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_type="azure",
)

/tmp/ipython-input-1008584197.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specifi

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(" You are a helpful assitant. \n User: { user_input}\n Assistant:")
print(prompt_template)

input_variables=[' user_input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[' user_input'], input_types={}, partial_variables={}, template=' You are a helpful assitant. \n User: { user_input}\n Assistant:'), additional_kwargs={})]


In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

/tmp/ipython-input-960525285.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [ ]:
while True:
  user_input = input("You: ")
  if user_input == "exit":
    break
  response = chain.run(user_input)
  print("Assistant: ", response)

You: Tell me about delhi
Assistant:  Certainly! Delhi, officially known as the National Capital Territory of Delhi (NCT), is one of the most vibrant and historically rich cities in India. It serves as the political, cultural, and administrative hub of the country. Here are some key aspects of Delhi:

---

### **Geography and Location**
- **Location:** Delhi is located in northern India and sits on the banks of the Yamuna River. It shares its borders with Haryana and Uttar Pradesh.
- **Area:** The NCT covers an area of approximately 1,484 square kilometers.
- **Climate:** Delhi has an extreme climate with hot summers, cold winters, and a monsoon season. Summers can reach temperatures above 40°C (104°F), while winters can dip below 5°C (41°F).

---

### **History**
Delhi has a rich history that spans thousands of years. It has been the center of several powerful empires and has seen the rise and fall of many dynasties:
- **Ancient Delhi:** Delhi is believed to be the site of Indraprastha

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

print(messages)
Output = llm.invoke(messages)
print(Output)
print(Output.content)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--cb8d949c-9746-4520-a846-d7bd247793f4-0'
The capital of France is **Paris**.


In [ ]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one for you:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--1efb12fc-9f04-4eec-bf05-c1b610ca41f1-0'
CPU times: user 5.07 ms, sys: 0 ns, total: 5.07 ms
Wall time: 409 ms


In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [ ]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one:\n\nWhy don't skeletons fight each other?\n\nBecause they don't have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--a67ec375-ff38-4511-977d-b3dee8d148ce-0'
CPU times: user 5.3 ms, sys: 38 µs, total: 5.34 ms
Wall time: 430 ms


In [ ]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one:\n\nWhy don't skeletons fight each other?\n\nBecause they don't have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--a67ec375-ff38-4511-977d-b3dee8d148ce-0' usage_metadata={'total_cost': 0}
CPU times: user 755 µs, sys: 0 ns, total: 755 µs
Wall time: 747 µs


In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's a light-hearted joke for you:\n\nWhy don’t skeletons fight each other?  \nBecause they don’t have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 11, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--91d04b8d-1a30-4e8a-8926-3dcae025780f-0'
CPU times: user 22.4 ms, sys: 1.16 ms, total: 23.5 ms
Wall time: 591 ms


In [ ]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's a light-hearted joke for you:\n\nWhy don’t skeletons fight each other?  \nBecause they don’t have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 11, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--91d04b8d-1a30-4e8a-8926-3dcae025780f-0' usage_metadata={'total_cost': 0}
CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 2.74 ms


In [ ]:
%%time
prompt = 'write a rock song about moon.'
response = llm.invoke(prompt)
print(response.content)

**Title: "Howlin' at the Moon"**

*(Verse 1)*  
Underneath the silver glow, it pulls me in, I lose control  
The night’s alive, the world stands still, chasing shadows over hills  
Up above, that mystic sphere, whispering secrets in my ear  
A cosmic dance, a timeless tune, my heart is howlin' at the moon  

*(Pre-Chorus)*  
Oh, it's calling, calling my name  
A wild desire I can't contain  
Through the darkness, I rise, I swoon  
Forever bound to the light of the moon  

*(Chorus)*  
Howlin' at the moon tonight  
Let it take me, let it guide my fight  
A rebel’s heart in the lunar light  
Howlin' at the moon, feels so right  

*(Verse 2)*  
Crater scars, its ancient face, a witness to this human race  
A tidal pull, a lover’s ache, it stirs the fire, it makes me quake  
In its glow, I find my truth, the kind of freedom I can’t lose  
A lonely wolf, a midnight croon, I'm alive beneath the moon  

*(Pre-Chorus)*  
Oh, it's burning, burning my soul  
A restless spirit out of control  
Th

In [ ]:
%%time
prompt = 'write a rock song about moon.'
response = llm.invoke(prompt)
print(response.content)

**Title: "Howlin' at the Moon"**

*(Verse 1)*  
Underneath the silver glow, it pulls me in, I lose control  
The night’s alive, the world stands still, chasing shadows over hills  
Up above, that mystic sphere, whispering secrets in my ear  
A cosmic dance, a timeless tune, my heart is howlin' at the moon  

*(Pre-Chorus)*  
Oh, it's calling, calling my name  
A wild desire I can't contain  
Through the darkness, I rise, I swoon  
Forever bound to the light of the moon  

*(Chorus)*  
Howlin' at the moon tonight  
Let it take me, let it guide my fight  
A rebel’s heart in the lunar light  
Howlin' at the moon, feels so right  

*(Verse 2)*  
Crater scars, its ancient face, a witness to this human race  
A tidal pull, a lover’s ache, it stirs the fire, it makes me quake  
In its glow, I find my truth, the kind of freedom I can’t lose  
A lonely wolf, a midnight croon, I'm alive beneath the moon  

*(Pre-Chorus)*  
Oh, it's burning, burning my soul  
A restless spirit out of control  
Th

In [ ]:
%%time
prompt = 'write a rock song about moon and sun.'
response = llm.invoke(prompt)
print(response.content)

**"Eclipse of Fire"**  
*(A Rock Anthem about the Moon and Sun)*  

---

**[Verse 1]**  
The moon whispers secrets in the dead of night,  
Silver shadows dance beneath its pale light.  
It’s a quiet rebel, cold and untamed,  
A lonely traveler with no one to blame.  

The sun roars loud, a burning king of fire,  
Rising high, it fuels the world's desire.  
Golden rays cut through the darkest skies,  
It’s the heart of life, where all hope lies.  

---

**[Pre-Chorus]**  
Two forces collide, pulling on my soul,  
Caught in the gravity, losing control.  

---

**[Chorus]**  
Moon and sun, locked in a cosmic fight,  
Day turns to darkness, and shadows ignite.  
Eclipse of fire, a dance of despair,  
We’re caught in the crossfire, gasping for air.  

---

**[Verse 2]**  
The moon’s a poet, spinning songs of the past,  
Its light fades slowly, but its beauty lasts.  
The sun’s a warrior, blazing trails up high,  
It rules the heavens, never asking why.  

They meet in silence, an eternal ch

In [ ]:
%%time
prompt = 'write a rock new song about moon and sun.'
for chunk in llm.stream(prompt):
  print(chunk.content, end="", flush=True)

**Title: "Chasing the Moon, Dancing with the Sun"**

*(Verse 1)*  
I’ve been running through the shadows, trying to find my light,  
Caught between the darkness and the fire burning bright.  
The moon whispers secrets in the stillness of the night,  
The sun pulls me closer, sets my soul alight.  

*(Pre-Chorus)*  
Oh, they’re pulling me apart, but I can’t turn away,  
Chasing the moon, while the sun begs me to stay.  

*(Chorus)*  
Moon and sun, they collide in my mind,  
One’s a quiet dream, the other’s wild and untamed.  
I’m chasing the moon, dancing with the sun,  
Caught in their orbit, I’ll never outrun.  

*(Verse 2)*  
Silver shadows guide me where the moonlight breaks,  
But the golden rays of morning leave me wide awake.  
They’re opposites, they’re rivals, yet they share the same sky,  
And I’m stuck somewhere in the middle, asking myself why.  

*(Pre-Chorus)*  
Oh, they tear me in two, but I’ll never let go,  
Chasing the moon, as the sun steals the show.  

*(Chorus)*  


In [ ]:
template = ''' You are a experienced machine learinig engineer.
choose which type of problem is this {problem} is based on data and desire result in {language}.'''

prompt_template = prompt_template.from_template(template)

prompt = prompt_template.format(problem="you have to predict how much icecrea will sale in this festival sale", language="hindi")
print(prompt)


Human:  You are a experienced machine learinig engineer.
choose which type of problem is this you have to predict how much icecrea will sale in this festival sale is based on data and desire result in hindi.


In [ ]:
output = llm.invoke(prompt)
print(output.content)

This type of problem falls under **Regression** in machine learning. Regression problems are used when the goal is to predict a continuous numerical value, such as sales, prices, or temperatures, based on input data.

Here’s how it applies to your situation:

- **Problem Type:** Predicting how much ice cream will sell during a festival.
- **Input Data:** Historical data, such as weather conditions, previous festival sales, date, time, location, etc.
- **Output:** A continuous numerical value (amount of ice cream sales).
- **Desired Result in Hindi:** अनुमान लगाना कि इस त्यौहार में कितनी आइसक्रीम बिकेगी।

### Suitable Algorithms:
You can use regression algorithms like:
- Linear Regression
- Decision Tree Regression
- Random Forest Regression
- Gradient Boosting (e.g., XGBoost, LightGBM)
- Neural Networks (if the data is complex)

Let me know if you need guidance on implementing this! 😊


In [ ]:
from langchain.prompts import HumanMessagePromptTemplate , ChatMessagePromptTemplate
from langchain_core.messages import SystemMessage


chat_template = ChatPromptTemplate.from_messages(
   [
       SystemMessage(content="You responsed only in JSON Format."),
       HumanMessagePromptTemplate.from_template("{user_input}")
   ]
)

chat_prompt = chat_template.format_prompt(user_input="Top 5 countries in world by population.").to_messages()
print(chat_prompt)



[SystemMessage(content='You responsed only in JSON Format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 5 countries in world by population.', additional_kwargs={}, response_metadata={})]


In [ ]:
output = llm.invoke(chat_prompt)
print(output.content)

```json
{
  "top_5_countries_by_population": [
    {
      "rank": 1,
      "country": "China",
      "population": "1,425,000,000"
    },
    {
      "rank": 2,
      "country": "India",
      "population": "1,421,000,000"
    },
    {
      "rank": 3,
      "country": "United States",
      "population": "336,000,000"
    },
    {
      "rank": 4,
      "country": "Indonesia",
      "population": "277,000,000"
    },
    {
      "rank": 5,
      "country": "Pakistan",
      "population": "240,000,000"
    }
  ]
}
```


In [ ]:
from re import VERBOSE
from langchain.chains import LLMChain

template = ''' You are a experienced machine learinig engineer.
choose which type of problem is this {problem} is based on data and desire result in {language}.'''

prompt_template = prompt_template.from_template(template)

chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

output = chain.invoke({"problem":"you have to predict how much icecrea will sale in this festival sale", "language":"hindi"})
print(output)




> Entering new LLMChain chain...
Prompt after formatting:
Human:  You are a experienced machine learinig engineer.
choose which type of problem is this you have to predict how much icecrea will sale in this festival sale is based on data and desire result in hindi.

> Finished chain.
{'problem': 'you have to predict how much icecrea will sale in this festival sale', 'language': 'hindi', 'text': 'यह समस्या एक **Regression Problem** है। \n\nक्योंकि यहाँ आपको ice cream की बिक्री (sale) की मात्रा को एक संख्या या मूल्य के रूप में भविष्यवाणी करनी है। Regression का उपयोग तब किया जाता है जब हमें किसी continuous value (जैसे बिक्री की मात्रा, तापमान, कीमत आदि) का अनुमान लगाना हो। \n\nडेटा और त्योहारी बिक्री के पैटर्न के आधार पर आप इस समस्या को हल करने के लिए regression models जैसे Linear Regression, Decision Trees, Random Forest, या Gradient Boosting Algorithms (जैसे XGBoost, LightGBM) का उपयोग कर सकते हैं। \n\nअगर डेटा में समय के साथ बिक्री का पैटर्न हो तो आप Time Series Analysis भी कर सकते ह

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm | StrOutputParser()

output = chain.invoke({"problem":"you have to predict how much icecrea will sale in this festival sale", "language":"hindi"})
print(output)

यह समस्या एक **Regression Problem** है। 

क्योंकि यहाँ आपको ice cream की बिक्री (sale) की मात्रा को एक संख्या या मूल्य के रूप में भविष्यवाणी करनी है। Regression का उपयोग तब किया जाता है जब हमें किसी continuous value (जैसे बिक्री की मात्रा, तापमान, कीमत आदि) का अनुमान लगाना हो। 

डेटा और त्योहारी बिक्री के पैटर्न के आधार पर आप इस समस्या को हल करने के लिए regression models जैसे Linear Regression, Decision Trees, Random Forest, या Gradient Boosting Algorithms (जैसे XGBoost, LightGBM) का उपयोग कर सकते हैं। 

अगर डेटा में समय के साथ बिक्री का पैटर्न हो तो आप Time Series Analysis भी कर सकते हैं।


In [ ]:
from langchain.chains import LLMChain , SimpleSequentialChain

prompt_template_1 = prompt_template.from_template(
    template = 'You are an expereined data scientist and Python Programer. Write a function that implement the concpet of {concept}.'
    )

chain_1 = LLMChain(llm=llm, prompt=prompt_template_1)

llm_2 = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_KEY,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_type="azure",
    temperature=0.9
)

prompt_template_2 = prompt_template.from_template(
    template = 'Write a python function that implement the concpet of {concept}.'
    )

chain_2 = LLMChain(llm=llm_2, prompt=prompt_template_2)

overall = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

output = overall.invoke("Linear Regression")
print(output)



/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co



> Entering new SimpleSequentialChain chain...
Certainly! Below is a Python implementation of a simple Linear Regression model using NumPy. We'll write a function that calculates the coefficients \( m \) (slope) and \( b \) (intercept) for fitting a straight line \( y = mx + b \) to a given dataset of \( x \) and \( y \).

### Linear Regression Function
```python
import numpy as np

def linear_regression(x, y):
    """
    Perform Linear Regression to find the slope (m) and intercept (b) of the line y = mx + b.

    Parameters:
    x (array-like): Independent variable (input features).
    y (array-like): Dependent variable (target values).

    Returns:
    tuple: (m, b) where m is the slope and b is the intercept.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)

    # Calculate the mean of x and y
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    # Calculate the slope (m)
    numerator = np.sum((x - x_mean) * (y - y_mean))  # Covariance betwe

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate , HumanMessagePromptTemplate
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser

llm = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_KEY,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_type="azure",
    temperature=0.9
)


prompt = ChatPromptTemplate.from_messages(
   [
       SystemMessage(content="You responsed only in English."),
       HumanMessagePromptTemplate.from_template("{user_input}")
   ]
)


chain = prompt | llm | StrOutputParser()

while True:
  user_input = input("You: ")
  if user_input == "exit":
    break
  output = chain.invoke({"user_input":user_input})
  print("Assistant: ", output)
  print("_" * 50)




/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co

You: tell me about patna
Assistant:  Patna is the capital city of Bihar, a state in eastern India, and is one of the oldest continuously inhabited cities in the world. It has a rich history, significant cultural heritage, and serves as a major educational, political, and economic center.

### Historical Significance:
- **Ancient Patliputra:** Patna was historically known as **Patliputra**, which was the capital of several powerful empires in ancient India, including the Maurya, Gupta, and Shunga dynasties. It played a crucial role in Indian history as a center of politics, administration, education, and culture.
- **Buddhism and Jainism:** The region holds great importance in the history of Buddhism and Jainism, as both religions flourished in and around Patna. It is close to significant religious sites like Bodh Gaya, Rajgir, and Vaishali.
- **Historical Events:** Greek historians like Megasthenes described the grandeur of ancient Patliputra during the Mauryan Empire.

### Modern Patn

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder



In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/tmp/ipython-input-1496473724.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [ ]:
while True:
  user_input = input("You: ")
  if user_input.lower() in ["exit" , "quit", "bye"] :
    print("Goodbye")
    break
  output = chain.invoke({"user_input":user_input})
  print("Assistant: ", output)
  print("_" * 50)

You: Hi
Assistant:  {'user_input': 'Hi', 'chat_history': [HumanMessage(content='Tell me about sun', additional_kwargs={}, response_metadata={}), AIMessage(content='The Sun is the star at the center of our solar system. It\'s a massive, nearly perfect sphere of hot plasma that generates energy through nuclear fusion. Its size, gravitational pull, and energy output make it the most influential celestial body in our solar system, shaping everything from the orbits of planets to Earth\'s climate. Here\'s a deeper look at the Sun:\n\n### **Basic Facts About the Sun**\n1. **Type of Star**: The Sun is classified as a G-type main-sequence star (G2V), often referred to as a "yellow dwarf." However, it actually emits white light that appears yellow from Earth due to atmospheric scattering.\n2. **Diameter**: Approximately 1.39 million kilometers (about 864,000 miles), making it about 109 times wider than Earth.\n3. **Mass**: Roughly 1.989 × 10^30 kilograms, which is about 333,000 times the mass o

In [ ]:
# how to print the content instant of json format


while True:
  user_input = input("You: ")
  if user_input.lower() in ["exit" , "quit", "bye"] :
    print("Goodbye")
    break
  output = chain.run({"user_input":user_input})
  print("Assistant: ", output)
  print("_" * 50)

You: write about crona virus in paragraph
Assistant:  The coronavirus, officially known as COVID-19, is a highly contagious viral infection that emerged in late 2019 in the city of Wuhan, China. Classified as a novel coronavirus, it quickly spread worldwide, leading to a global pandemic declared by the World Health Organization (WHO) in March 2020. The virus primarily spreads through respiratory droplets and close contact, with symptoms ranging from mild, such as fever, cough, and fatigue, to severe, including difficulty breathing and complications like pneumonia. Vulnerable populations, such as the elderly and those with underlying health conditions, faced increased risks of severe outcomes. The pandemic had profound impacts on global health systems, economies, and daily life, prompting widespread lockdowns, travel restrictions, and shifts to remote work and education. In response, scientists and pharmaceutical companies rapidly developed vaccines, which became a crucial tool in comba

KeyboardInterrupt: Interrupted by user

In [ ]:
from langchain.memory import FileChatMessageHistory

histroy = FileChatMessageHistory("chat_history.json")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True ,
    chat_memory=histroy

)


chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)


while True:
  user_input = input("You: ")
  if user_input.lower() in ["exit" , "quit", "bye"] :
    print("Goodbye")
    break
  output = chain.invoke({"user_input":user_input})
  print("Assistant: ", output)
  print("_" * 50)


You: Hi
Assistant:  {'user_input': 'Hi', 'chat_history': [HumanMessage(content='Tell me about sun', additional_kwargs={}, response_metadata={}), AIMessage(content='The Sun is the star at the center of our solar system. It\'s a massive, nearly perfect sphere of hot plasma that generates energy through nuclear fusion. Its size, gravitational pull, and energy output make it the most influential celestial body in our solar system, shaping everything from the orbits of planets to Earth\'s climate. Here\'s a deeper look at the Sun:\n\n### **Basic Facts About the Sun**\n1. **Type of Star**: The Sun is classified as a G-type main-sequence star (G2V), often referred to as a "yellow dwarf." However, it actually emits white light that appears yellow from Earth due to atmospheric scattering.\n2. **Diameter**: Approximately 1.39 million kilometers (about 864,000 miles), making it about 109 times wider than Earth.\n3. **Mass**: Roughly 1.989 × 10^30 kilograms, which is about 333,000 times the mass o

** different for each LLM provider **

*   library required
*   secrets key
*   configure LLM connection

** common for all LLM **

*   prompt template / schema / chatmesage
*   chain ( verbose= True)
*   call the chain or llm
*   cache memoery
*   streaming
*   simplesequenticalchain
*   stroputput
*   memeory



In [ ]:
text = '''
The coronavirus, officially known as COVID-19, is a highly contagious viral infection that emerged in late 2019 in the city of Wuhan, China.
Classified as a novel coronavirus, it quickly spread worldwide, leading to a global pandemic declared by the World Health Organization
(WHO) in March 2020. The virus primarily spreads through respiratory droplets and close contact, with symptoms ranging from mild, such as
fever, cough, and fatigue, to severe, including difficulty breathing and complications like pneumonia. Vulnerable populations, such as
the elderly and those with underlying health conditions, faced increased risks of severe outcomes. The pandemic had profound impacts on
global health systems, economies, and daily life, prompting widespread lockdowns, travel restrictions, and shifts to remote work and
education. In response, scientists and pharmaceutical companies rapidly developed vaccines, which became a crucial tool in combating
the virus and reducing its spread. Despite progress in managing the pandemic, the emergence of new variants posed ongoing challenges,
underscoring the importance of public health measures, vaccination campaigns, and global cooperation to mitigate the virus's impacts.'''

messages = [
  SystemMessage(content="You are an expert copywriter the expertize in summarizing documents."),
  HumanMessage(content=f' please provide a short and concise summary of the following text : \n TEXT :  {text}')
]

output = llm.invoke(messages)
print(output.content)

COVID-19, a highly contagious novel coronavirus, emerged in Wuhan, China, in late 2019 and rapidly spread globally, leading the WHO to declare a pandemic in March 2020. Transmitted through respiratory droplets, the virus caused symptoms ranging from mild to severe, with vulnerable populations facing heightened risks. The pandemic disrupted health systems, economies, and daily life, prompting lockdowns, travel restrictions, and remote work. Vaccines were rapidly developed as a key tool to combat the virus, but new variants continued to challenge global efforts, highlighting the need for public health measures, vaccination, and cooperation.


In [ ]:
llm.get_num_tokens(text)

230

In [ ]:
from langchain.chains import load_summarize_chain
from langchain.docstore import document

In [ ]:
from google.colab import files
uploaded = files.upload()

with open ('sj.txt', encoding = 'utf-8') as f:
  text = f.read()

docs = [document.Document(page_content=text)]

print(text[:500])


Saving sj.txt to sj.txt
I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit.


In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ''' write a concise and short summary of the following text. \n TEXT : {text} '''

prompt = ChatPromptTemplate.from_template(template)

chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

output = chain.run(docs)

print(output)

Steve Jobs delivered a commencement speech sharing three impactful stories from his life. He spoke about connecting the dots, detailing how dropping out of college allowed him to explore his interests, such as calligraphy, which later influenced the Macintosh computer's typography. He addressed love and loss, recounting his firing from Apple, which led him to start NeXT and Pixar, ultimately bringing innovation and personal growth. Lastly, he reflected on death, emphasizing the finite nature of life and the importance of living authentically and following one’s heart. He urged graduates to pursue what they love and concluded with the advice: "Stay Hungry. Stay Foolish."


In [ ]:
# 128K [ Input token and output token]
#limit 10000

from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],
    chunk_size=10000,
    chunk_overlap=100
)

chunk = text_splitter.create_documents([text])


len(chunk)



2

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
output = chain.run(chunk)
print(output)

Steve Jobs' commencement speech shared three life lessons: trust in life's journey, find opportunity in setbacks, and embrace life's impermanence. Dropping out of college led him to discover calligraphy, shaping Macintosh's design. His firing from Apple inspired new ventures with NeXT and Pixar, ultimately fueling his return and innovation at Apple. A cancer diagnosis reminded him to live intentionally and authentically, urging graduates to pursue their passions and embrace individuality. He concluded with the advice: "Stay Hungry. Stay Foolish."


In [ ]:
from langchain.document_loaders import PyPDFLoader


In [ ]:
!pip install -q pdf2image pdfminer pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

pdf = PyPDFLoader('attention_is_all_you_need.pdf')
data = loader.load()

Saving attention_is_all_you_need.pdf to attention_is_all_you_need.pdf


ValueError: File path  is not a valid file or url